# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
# Data Science
import pandas as pd
import numpy as np

# API requests
import requests
import json
from pprint import pprint

#Data Visualization
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns

# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Greenville,35.6127,-77.3663,35.13,92,100,0.00,US,1733107207
1,1,Waitangi,-43.9535,-176.5597,57.81,75,86,11.01,NZ,1733107440
2,2,Luena,-11.7833,19.9167,60.67,93,13,1.36,AO,1733107441
3,3,George Town,5.4112,100.3354,84.13,69,20,8.05,MY,1733107442
4,4,Port-aux-Français,-49.3500,70.2167,38.48,78,98,7.11,TF,1733107443


In [3]:
city_data_df['Date'] = pd.to_datetime(city_data_df['Date'], unit='s')
city_data_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Greenville,35.6127,-77.3663,35.13,92,100,0.00,US,2024-12-02 02:40:07
1,1,Waitangi,-43.9535,-176.5597,57.81,75,86,11.01,NZ,2024-12-02 02:44:00
2,2,Luena,-11.7833,19.9167,60.67,93,13,1.36,AO,2024-12-02 02:44:01
3,3,George Town,5.4112,100.3354,84.13,69,20,8.05,MY,2024-12-02 02:44:02
4,4,Port-aux-Français,-49.3500,70.2167,38.48,78,98,7.11,TF,2024-12-02 02:44:03
...,...,...,...,...,...,...,...,...,...,...
588,588,Manaure,11.7751,-72.4445,82.09,80,65,15.12,CO,2024-12-02 02:56:34
589,589,Veigné,47.2856,0.7408,54.14,84,100,6.91,FR,2024-12-02 02:56:35
590,590,Puerto Colombia,10.9878,-74.9547,84.29,81,99,16.84,CO,2024-12-02 02:56:36
591,591,Geraldton,-28.7667,114.6000,72.01,53,75,18.41,AU,2024-12-02 02:56:37


In [4]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593 entries, 0 to 592
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   City_ID     593 non-null    int64         
 1   City        593 non-null    object        
 2   Lat         593 non-null    float64       
 3   Lng         593 non-null    float64       
 4   Max Temp    593 non-null    float64       
 5   Humidity    593 non-null    int64         
 6   Cloudiness  593 non-null    int64         
 7   Wind Speed  593 non-null    float64       
 8   Country     592 non-null    object        
 9   Date        593 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(3), object(2)
memory usage: 46.5+ KB


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 1000,
    frame_height = 500,
    color = "City",
    size="Humidity",
    hover_cols=["City", "Country", "Max Temp", "Humidity"], # Add 'city' to the tooltip
)

# Display the map plot
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
# Assuming Ideal weather conditions:

ideal_temp_min = 60  # Minimum temperature in Fahrenheit
ideal_temp_max = 80  # Maximum temperature in Fahrenheit
ideal_humidity_min = 40  # Minimum humidity in percentage
ideal_humidity_max = 60  # Maximum humidity in percentage
ideal_wind_max = 10  # Maximum wind speed in mph

# Drop any rows with null values
ideal_weather_df = city_data_df.dropna()

# Filter the DataFrame based on the ideal weather conditions

ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] >= ideal_temp_min) & 
    (city_data_df["Max Temp"] <= ideal_temp_max) &
    (city_data_df["Humidity"] >= ideal_humidity_min) & 
    (city_data_df["Humidity"] <= ideal_humidity_max) &
    (city_data_df["Wind Speed"] <= ideal_wind_max) 
]

# Display the filtered DataFrame with cities that meet the ideal weather conditions
ideal_weather_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,Mbaïki,3.8678,17.9892,70.43,53,28,2.01,CF,2024-12-02 02:44:07
48,48,Isla Vista,34.4133,-119.8610,64.45,40,0,3.29,US,2024-12-02 02:45:02
74,74,Masterton,-40.9597,175.6575,75.20,48,97,6.31,NZ,2024-12-02 02:45:33
79,79,Zemio,5.0314,25.1361,66.78,52,6,2.53,CF,2024-12-02 02:45:39
86,86,Huilong,31.8111,121.6550,63.00,48,0,6.17,CN,2024-12-02 02:45:48
121,121,Bhinga,27.7167,81.9333,63.93,54,0,3.06,IN,2024-12-02 02:46:33
153,153,La Barca,20.2833,-102.5667,62.85,42,0,1.97,MX,2024-12-02 02:47:14
182,182,Burao,9.5221,45.5336,63.01,60,5,5.37,SO,2024-12-02 02:47:52
238,238,Mpraeso,6.5932,-0.7346,67.89,56,0,2.26,GH,2024-12-02 02:49:05
251,251,Nakamura,32.9833,132.9333,65.91,54,1,1.66,JP,2024-12-02 02:49:21


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
hotel_rows = []

for index, row in ideal_weather_df.iterrows():
    # from the Weather CSV
    city = row["City"]
    country = row["Country"]
    latitude = row["Lat"]
    longitude = row["Lng"]
    humidity = row["Humidity"]

    # Set the parameters for the type of place
    categories = "accommodation.hotel"
    radius = 10000 # 10km
    
    # Set the parameters for the type of search
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    limit = 5
    
    # set up a parameters dictionary
    params = {
        "categories":categories,
        "limit":limit,
        "filter":filters,
        "bias":bias,
        "apiKey":geoapify_key    
    }
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Run request
    try:
        response = requests.get(base_url, params=params)
        # print(response.status_code)
        if (response.status_code == 200):
            data = response.json()
            
            # Print the results
            results = data.get("features", [])
            
            # Resiliency/Error Handling
            if len(results) > 0:
                place = results[0]
        
                # normalize data
                address = place.get("properties", {}).get("formatted")
                name = place.get("properties", {}).get("name")
    
            
                # return object
                hotel_row = {
                    "City": city,
                    "Country": country,
                    "Lat": latitude,
                    "Lng": longitude,
                    "Humidity" : humidity,
                    "Hotel Name": name,
    
                }
            else:
                hotel_row = {
                    "City": city,
                    "Country": country,
                    "Lat": latitude,
                    "Lng": longitude,
                    "Humidity": humidity,
                    "Hotel Name": None,
                }
               
        else:
            print(f"Error: {response.status_code}, {response.text}")
            hotel_row = {
                "City": city,
                "Country": country,
                "Lat": latitude,
                "Lng": longitude,
                "Humidity" : humidity,
                "Hotel Name": None,

            }
    except Exception as e:
        print(e)
        hotel_row = {
                "City": city,
                "Country": country,
                "Lat": latitude,
                "Lng": longitude,
                "Humidity" : humidity,
                "Hotel Name": None,

            }

    # append to hotel list
    hotel_rows.append(hotel_row)

In [17]:
hotel_df = pd.DataFrame(hotel_rows)
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Mbaïki,CF,3.8678,17.9892,53,None
1,Isla Vista,US,34.4133,-119.8610,40,The Club & Guest House
2,Masterton,NZ,-40.9597,175.6575,48,Copthorne Hotel Solway Park
3,Zemio,CF,5.0314,25.1361,52,None
4,Huilong,CN,31.8111,121.6550,48,先豪国际酒店
5,Bhinga,IN,27.7167,81.9333,54,None
6,La Barca,MX,20.2833,-102.5667,42,Hotel Real de La Barca
7,Burao,SO,9.5221,45.5336,60,Liiban hotel burco ( ENG:Shibiin )
8,Mpraeso,GH,6.5932,-0.7346,56,Ohene Nana Classic Hotel
9,Nakamura,JP,32.9833,132.9333,54,ホテル アイランド


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 10000  # 10km radius
params = {
    "categories": "accommodation.hotel",
    "limit": 1,  # Limit to 1 result (nearest hotel)
    "apiKey": geoapify_key  # Corrected key parameter
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL for the Geoapify API
    base_url = "https://api.geoapify.com/v2/places"

    # Make the API request using the params dictionary
    try:
        response = requests.get(base_url, params=params)

        # Convert the API response to JSON format
        name_address = response.json()


        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        if "features" in name_address and len(name_address["features"]) > 0:
            hotel = name_address["features"][0]  # Get the first hotel            
            # Try to get the hotel name
            hotel_name = hotel['properties'].get('name', 'No hotel name available')
            hotel_df.loc[index, "Hotel Name"] = hotel_name
        else:
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    except requests.exceptions.RequestException as e:
        print(f"Error with the API request for {row['City']}: {e}")
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display the updated hotel_df with hotel names
hotel_df



Starting hotel search
Mbaïki - nearest hotel: No hotel found
Isla Vista - nearest hotel: The Club & Guest House
Masterton - nearest hotel: Copthorne Hotel Solway Park
Zemio - nearest hotel: No hotel found
Huilong - nearest hotel: 先豪国际酒店
Bhinga - nearest hotel: No hotel found
La Barca - nearest hotel: Hotel Real de La Barca
Burao - nearest hotel: Liiban hotel burco ( ENG:Shibiin )
Mpraeso - nearest hotel: Ohene Nana Classic Hotel
Nakamura - nearest hotel: ホテル アイランド
Puente Alto - nearest hotel: Hotel Mary Louis
Yenangyaung - nearest hotel: No hotel found
Wenchang - nearest hotel: No hotel found
Udaipura - nearest hotel: No hotel found
Naze - nearest hotel: ビジネス旅館畠山
Kerikeri - nearest hotel: Avalon Resort
Al Muwayh - nearest hotel: فندق ومطعم الموية
Bulawayo - nearest hotel: Selboune Hotel
Nishifukuma - nearest hotel: 割烹旅館 正直亭
Ārvi - nearest hotel: No hotel found
Dakhla - nearest hotel: Hotel Ayouz
Charthāwal - nearest hotel: No hotel found
Wej - nearest hotel: فندق رالين 2
Mogok - neares

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Mbaïki,CF,3.8678,17.9892,53,No hotel found
1,Isla Vista,US,34.4133,-119.8610,40,The Club & Guest House
2,Masterton,NZ,-40.9597,175.6575,48,Copthorne Hotel Solway Park
3,Zemio,CF,5.0314,25.1361,52,No hotel found
4,Huilong,CN,31.8111,121.6550,48,先豪国际酒店
5,Bhinga,IN,27.7167,81.9333,54,No hotel found
6,La Barca,MX,20.2833,-102.5667,42,Hotel Real de La Barca
7,Burao,SO,9.5221,45.5336,60,Liiban hotel burco ( ENG:Shibiin )
8,Mpraeso,GH,6.5932,-0.7346,56,Ohene Nana Classic Hotel
9,Nakamura,JP,32.9833,132.9333,54,ホテル アイランド


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 800,
    frame_height = 600,
    color = "City",
    size="Humidity",
    hover_cols=["City", "Country", "Max Temp", "Humidity", "Hotel Name", "Country"],
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name,Country)